# Pre-process Text

## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
import re
from sklearn.model_selection import train_test_split
import random

## Functions

### Helper Functions - Data Cleaning

In [2]:
def _remove_urls(tweets, replace_token='<URL>'):
    """ Replace URLs in text with a replacement token
    Args:
        tweets (list): Tweets to clean
        replace_token (str): String to replace URL
    Returns:
        clean_tweets (list): Cleaned tweet list
        removed (list): List of Removed Tweets
    """
    removed = []
    clean_tweets = []
    for tweet in tweets:
        try:
            clean_tweet = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b',
                          replace_token, tweet, flags=re.MULTILINE)
            clean_tweets.append(clean_tweet)
        except TypeError:
            removed.append(tweet)
    return clean_tweets, removed

In [3]:
def _remove_RT(tweets):
    """ Remove Retweets
    Args:
        tweets (list): Tweets to clean
    Returns:
        clean_tweets (list): Cleaned tweet list
        removed (list): List of Removed Tweets
    """
    removed = []
    clean_tweets = []
    for tweet in tweets:
        try:
            if 'RT' ==  tweet.split()[0]:
                removed.append(tweet)
            else: clean_tweets.append(tweet)
        except (TypeError, AttributeError):
            pass
    return clean_tweets, removed

### Helper Functions - File Read/Write

In [4]:
def _txt_file_to_sents(path, debug=True):
    """ (Helper) Convert txt file into a list of sentences
    Args:
        path (str): Path to txt file
        debug (bool): Whether to print information
    Returns:
        sents (list): List of sentences in corpus
    """
    f = open(path, "r", errors="ignore")
    text = f.read()
    f.close()
    #nltk.download('punkt')
    sents = nltk.sent_tokenize(text)
    if debug: print("Converted", path.split('/')[-1],"| Sentences: ", len(sents))
    return sents

In [5]:
def _sents_to_txt(out_path, sents, debug=True, new_line=True):
    """ (Helper) Write an array of sentences to a text file
    Args:
        out_path (str): Path to write txt file
        sents (list): List of sentences in corpus
        debug (bool): Whether to print information
        new_line (bool): Whether to add a newline char to sentences
    """
    fail_sents = []
    with open(out_path,'w') as out_file:
        for sentence in sents:
            sentence = str(sentence)
            try:
                sentence = sentence+'\n' if new_line else sentence
                out_file.write(sentence)
            except UnicodeEncodeError:
                fail_sents.append(sentence)
    if debug:
        print("Num Sentences Failed to Write: ", len(fail_sents))
        #for i in fail_sents: print(i)

In [6]:
def _sents_to_train_dev_txt(sents, file_name, debug=True, new_line=True, random_state=42,
                           returns=False, test_size=0.12):
    """ (Helper) Convert a list of sentences to a train and dev set
    Args:
        sents (list): List of sentences in corpus
        file_name (str): "positive" or "negative"
        debug (bool): Whether to print information
        new_line (bool): Whether to add a newline char to sentences
        random_state (int): random state for data split
        returns (bool): Whether to return train and dev sets
        test_size (float): Percentage in dev set
    Returns:
        train (list): Train set
        dev (list): Dev set
    """
    train, dev, _, _ = train_test_split(sents, np.zeros(len(sents)),
                                            test_size=test_size, 
                                        random_state=random_state)
    if debug: print("Num Train:", len(train), "| Num Dev:", len(dev))
    _sents_to_txt("train/" + file_name + ".txt", train, debug=debug, new_line=new_line)
    _sents_to_txt("dev/" + file_name + ".txt", dev, debug=debug, new_line=new_line)
    if returns: return train, dev

In [7]:
def _gen_full_text_txt():
    base_path = '../yelp/'
    paths = ['train/positive.txt', 'train/negative.txt', 'dev/positive.txt', 'dev/negative.txt']
    sents = []
    for path in paths:
        f = open(base_path + path, "r", errors="ignore")
        text = f.readlines()
        sents.append(text)
        f.close()
    _sents_to_txt("../yelp/full_text.txt", sents)
    

### Generate Original Dataset Sentences

#### English Data

In [8]:
from nltk.corpus import brown, gutenberg, inaugural
def gen_en_text_orig():
    """ Generate the orig en_text.txt file in the orig folder
    """
    en_text = open('orig/en_text_NEW.txt', 'w')
    # Gutenberg
    for file_id in gutenberg.fileids():
        if file_id == 'bible-kjv.txt' or 'shakespeare' in file_id:
            continue
        file_sents = gutenberg.sents(file_id)
        for sent in file_sents:
            new_sent = ' '.join(sent)
            en_text.write(new_sent + '\n')
    # Brown
    for file_id in brown.fileids():
        file_sents = brown.sents(file_id)
        for sent in file_sents:
            new_sent = ' '.join(sent)
            en_text.write(new_sent + '\n')
    # Inaugural
    for file_id in brown.fileids():
        if file_id == '2017-Trump.txt':
            continue
        file_sents = brown.sents(file_id)
        for sent in file_sents:
            new_sent = ' '.join(sent)
            en_text.write(new_sent + '\n')
    en_text.close()

### Clean Original Dataset Sentences

#### Trump Data

In [33]:
def _post_process_tweets(tweets, debug=True):
    count_removed = 0
    processed_tweets = []
    for tweet in tweets:
        if tweet[:7] == 'Twitter':
            try:
                source,text,created_at,retweet_count,favorite_count,is_retweet,id_str = tweet.split(',')
                processed_tweets.append(text)
                count_removed += 1
            except:
                processed_tweets.append(tweet)
                if debug:
                    print('Added to Dataset: %s' %tweet)
    if debug:
        print('Number of CSV lines post-processed: %d' %count_removed)
    return processed_tweets

def _clean_tweets(tweets, debug=True):
    """ Main Helper function to clean Tweets
    Args:
        tweets (list): Tweets to clean
        debug (bool): Whether to print information
    Returns:
        tweets (list): Cleaned tweet list
    """
    tweets, del_url = _remove_urls(tweets)
    tweets, del_RT = _remove_RT(tweets)
    if debug:
        print("Del Tweets (URL processing):", len(del_url))
        print("Del Tweets (RT processing):", len(del_RT))
    return tweets
    
def _gen_trump_sents(debug=True):
    """ Helper function to generate Trump data sentences
    Args:
        debug (bool): Whether to print information
    """
    rally_speeches = _txt_file_to_sents("orig/trump_10_2016_rally_speeches_orig.txt", debug=debug)
    other_speeches = _txt_file_to_sents("orig/trump_speeches.txt", debug=debug)
    tweet = pd.read_csv("orig/trump_tweets_orig.csv")
    tweet.columns = ['source','text','created_at','retweet_count','favorite_count','is_retweet','id_str']
    raw_tweets = list(tweet.text)
    clean_tweets = _clean_tweets(raw_tweets, debug=debug)
    processed_tweets = _post_process_tweets(clean_tweets, debug=debug)
    sents = rally_speeches + other_speeches + processed_tweets
    if debug: print("Num Sentences:", len(sents))
    return sents

def gen_trump_train_dev(debug=True, new_line=False, random_state=42, returns=True, test_size=0.12):
    """ Main function to generate train and dev set for Trump Data
    Args:
        debug (bool): Whether to print information
        new_line (bool): Whether to add a newline char to sentences
        random_state (int): random state for data split
        returns (bool): Whether to return train and dev sets
        test_size (float): Percentage in dev set
    Returns:
        train (list): Train set
        dev (list): Dev set
    """
    sents = _gen_trump_sents(debug=debug)
    trump_all = open('orig/trump_all_data.txt', 'w')
    for sent in sents:
        trump_all.write(sent + '\n')
    train, dev = _sents_to_train_dev_txt(sents, file_name="negative", debug=debug, new_line=new_line,
                                        random_state=random_state, returns=returns, test_size=test_size)
    if returns: return train, dev

#### English Data

In [10]:
def _gen_en_sents(k = 62399):
    """ Helper function to generate subsample of en sentences
    Args:
        k (int): Subsample size
    """
    gen_en_text_orig()
    f = open("orig/en_text_NEW.txt", "r")
    sents = f.readlines()
    f.close()
    return random.sample(sents, k)

def gen_en_train_dev(k, debug=True, new_line=False, random_state=42, returns=True, test_size=0.12):
    """ Main function to generate train and dev set for En Data
    Args:
        k (int): Subsample size
        debug (bool): Whether to print information
        new_line (bool): Whether to add a newline char to sentences
        random_state (int): random state for data split
        returns (bool): Whether to return train and dev sets
        test_size (float): Percentage in dev set
    Returns:
        train (list): Train set
        dev (list): Dev set
    """
    sents = _gen_en_sents(k = k)
    if debug: print("Num Sentences:", len(sents))
    train, dev = _sents_to_train_dev_txt(sents, file_name="positive", debug=debug, test_size=test_size,
                                         random_state=random_state, new_line=new_line, returns=returns)
    if returns: return train, dev

## Workflow

In [34]:
trump_train, trump_dev = gen_trump_train_dev(debug=True)

Converted trump_10_2016_rally_speeches_orig.txt | Sentences:  16503
Converted trump_speeches.txt | Sentences:  7695
Del Tweets (URL processing): 1
Del Tweets (RT processing): 2152
a
Added to Dataset: Twitter should let the banned Conservative Voices back onto their platform without restriction. It’s called Freedom of Speech remember. You are making a Giant Mistake!
a
Added to Dataset: Twitter has removed many people from my account and more importantly they have seemingly done something that makes it much harder to join - they have stifled growth to a point where it is obvious to all. A few weeks ago it was a Rocket Ship now it is a Blimp! Total Bias?
a
Added to Dataset: Twitter “SHADOW BANNING” prominent Republicans. Not good. We will look into this discriminatory and illegal practice at once! Many complaints.
a
Added to Dataset: Twitter is getting rid of fake accounts at a record pace. Will that include the Failing New York Times and propaganda machine for Amazon the Washington Post 

In [12]:
en_train, en_dev = gen_en_train_dev(k = 62399, debug=True)

Num Sentences: 62399
Num Train: 54911 | Num Dev: 7488
Num Sentences Failed to Write:  0
Num Sentences Failed to Write:  0
